# Evaluation Retrival

Um ganze sätze zu prüfen und zu vergleichen wird ebenfalls ein Cosinus-Similarity genutzt. Hierzu werden die Sätze in Vektoren umgewandelt und dann verglichen.

Die Testdaten wurden von Claude Code anhand der Produktdatei generiert. Sinnvoll wäre es, mit dem Kunden zu sprechen und echte Fragen zu sammeln.

In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas
import tqdm as tqdm
import json

/home/bruce/.local/share/mise/installs/python/3.13.7/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
temp_queries = [
      "Temperaturalarm bei Störung",           # Direkt aus Specs
      "Warnung bei Temperaturabweichung",
      "Optischer und akustischer Alarm",       # Aus Specs
      "SmartMonitoring Benachrichtigung",      # Aus Description  
      "E-Mail Warnung bei Temperaturproblem"   # Aus Description
  ]

size_queries = [
    "Außenmaße des Geräts",                  # Direkt aus Specs
    "Wie groß ist der Kühlschrank?",
    "Abmessungen in Zentimetern",            # Specs: "59,7 x 65,4 x 188,4" 
    "BxTxH Maße",
    "Platzbedarf des Medikamentenkühlschranks"
]

connectivity_queries = [
    "USB-Schnittstelle zum Datenauslesen",   # Direkt aus Description
    "WiFi LAN Verbindung",                   # Aus Specs: "LAN / WiFi integriert"
    "Netzwerk-Anschluss verfügbar",
    "Datenlogger per USB auslesen",          # Aus Specs
    "SmartModule Vernetzbarkeit"             # Aus Specs
]

energy_queries = [
    "Stromverbrauch pro Jahr",               # Specs: "172 kWh"
    "Energieverbrauch in 365 Tagen",         # Direkt aus Specs
    "Wie viel kWh verbraucht das Gerät?",
    "Betriebskosten Strom",
    "Ultra-efficient Energieeffizienz"      # Aus Description
]

safety_queries = [
    "Was passiert bei Stromausfall?",        # Description hat ganzen Abschnitt
    "Netzausfallalarm verfügbar",           # Specs: "unmittelbar bei Netzunterbrechung"
    "Batteriegepufferter Alarm",            # Description
    "SafetyDevice Schutzfunktion",          # Description
    "SmartLock automatische Verriegelung"   # Description
]

capacity_queries = [
    "Rauminhalt des Kühlschranks",          # Specs: "394 / 235 l"
    "Brutto Netto Liter",                   # Direkt aus Specs  
    "Wie viel passt in den Kühlschrank?",
    "Volumen für Medikamente",
    "Anzahl Schubfächer verfügbar"         # Specs: "8"
]

all_queries = temp_queries + size_queries + connectivity_queries + energy_queries + safety_queries + capacity_queries

## Chunks laden

In [3]:
descs = pandas.read_json('../2-chunking/chunks_description.jsonl', lines=True)
specs = pandas.read_json('../2-chunking/chunks_specs.jsonl', lines=True)
all_data = pandas.concat([descs, specs], ignore_index=True)

chunks = all_data['document'].tolist()

## Model laden und Chunks embedden

In [4]:
model = SentenceTransformer('deepset/gbert-large')

embeddings = model.encode(chunks, show_progress_bar=True, batch_size=16)

No sentence-transformers model found with name deepset/gbert-large. Creating a new one with mean pooling.
Batches: 100%|██████████| 289/289 [23:02<00:00,  4.78s/it] 


## Retrival

In [5]:
def evaluate_retrival(queries, embeddings, top_k=5):

    queries_embedded = model.encode(queries, show_progress_bar=True)
    similarities = util.cos_sim(queries_embedded, embeddings)

    results = []
    for i, query in enumerate(queries):
        
        query_similarity = similarities[i]
        top_indices = query_similarity.argsort(descending=True)[:top_k]

        for rank, chunk_idx in enumerate(top_indices):
            results.append({
                'query': query,
                'rank': rank + 1,
                'chunk_idx': chunk_idx.item(),
                'similarity': query_similarity[chunk_idx].item(),
                'chunk_text': chunks[chunk_idx.item()][:200],
                'metadata': all_data.iloc[chunk_idx.item()]['metadata']
            })

    return pandas.DataFrame(results)

In [6]:
results = evaluate_retrival(all_queries, embeddings)

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


## Top-K Ranking

In [7]:
comparsion = results.pivot_table(
    index='query',
    columns='rank',
    values='similarity',
    margins=True,
    margins_name='Mean'
)

comparsion

# bessere vergleiche verwenden. textuell statt similarity only.

rank,1,2,3,4,5,Mean
query,,,,,,
Abmessungen in Zentimetern,0.917942,0.910502,0.910502,0.910502,0.910502,0.911990
Anzahl Schubfächer verfügbar,0.959847,0.959847,0.956422,0.956422,0.950610,0.956630
Außenmaße des Geräts,0.932358,0.931788,0.931082,0.931082,0.931082,0.931478
Batteriegepufferter Alarm,0.872290,0.871800,0.869460,0.869460,0.869460,0.870494
Betriebskosten Strom,0.901588,0.898214,0.898214,0.898214,0.898214,0.898889
Brutto Netto Liter,0.903350,0.903350,0.903350,0.903350,0.903350,0.903350
BxTxH Maße,0.936077,0.935289,0.935289,0.935289,0.933737,0.935136
Datenlogger per USB auslesen,0.936467,0.936467,0.936467,0.936467,0.936467,0.936467
E-Mail Warnung bei Temperaturproblem,0.934055,0.934055,0.934055,0.933379,0.933323,0.933774
